# Instalação e Importe de Bibliotecas

In [ ]:
!pip install scikit-learn # Excluir do código no ambiente Git

In [ ]:
!pip install numpy pandas matplotlib seaborn # Excluir do código no ambiente Git

In [ ]:
!pip install streamlit # Excluir do código no ambiente Git

In [56]:
import streamlit as st
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import joblib
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import plotly.express as px

# Confirguração da página Streamlit e Importe | Treinamento do modelo

In [19]:
st.set_page_config(
    page_title="Análise Obesidade | Dashboard",
    layout="wide",
    initial_sidebar_state="expanded"
)

2025-07-10 14:22:34.781 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [57]:
dados = pd.read_csv("/content/Obesity.csv")
dados.head(3)

,Gender,Age,Height,Weight,family_history,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,Obesity
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight


In [5]:
X = dados.drop('Obesity', axis=1)
y = dados['Obesity']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=452457)

In [54]:
# Combine train and test sets before encoding to ensure consistent columns
X_combinado = pd.concat([X_train, X_test], ignore_index=True)
X_combinado_cod = pd.get_dummies(X_combinado, drop_first=True)

# Split the encoded data back into train and test sets
X_train_cod = X_combinado_cod.iloc[:len(X_train)]
X_test_cod = X_combinado_cod.iloc[len(X_train):]

# Verify the columns match (optional, but good practice)
print("Columns in X_train_encoded:", X_train_cod.columns.tolist())
print("Columns in X_test_encoded:", X_test_cod.columns.tolist())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definindo as features numéricas e categóricas
numeric_features = ['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']
categorical_features = ['Gender', 'family_history', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS']

Columns in X_train_encoded: ['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE', 'Gender_Male', 'family_history_yes', 'FAVC_yes', 'CAEC_Frequently', 'CAEC_Sometimes', 'CAEC_no', 'SMOKE_yes', 'SCC_yes', 'CALC_Frequently', 'CALC_Sometimes', 'CALC_no', 'MTRANS_Bike', 'MTRANS_Motorbike', 'MTRANS_Public_Transportation', 'MTRANS_Walking']
Columns in X_test_encoded: ['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE', 'Gender_Male', 'family_history_yes', 'FAVC_yes', 'CAEC_Frequently', 'CAEC_Sometimes', 'CAEC_no', 'SMOKE_yes', 'SCC_yes', 'CALC_Frequently', 'CALC_Sometimes', 'CALC_no', 'MTRANS_Bike', 'MTRANS_Motorbike', 'MTRANS_Public_Transportation', 'MTRANS_Walking']


## Utilizando o modelo LogisticRegression

In [40]:
modelo_lr = LogisticRegression(max_iter=2000)
modelo_lr.fit(X_train_cod, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=2000)

In [43]:
predicoes_lr = modelo_lr.predict(X_test_cod)

In [44]:
precisao_lr = accuracy_score(y_test, predicoes_lr)

## Utilizando o modelo RandomForestClassifier

In [45]:
modelo_rf = RandomForestClassifier()
modelo_rf.fit(X_train_cod, y_train)

RandomForestClassifier()

In [46]:
predicoes_rf = modelo_rf.predict(X_test_cod)

In [47]:
precisao_rf = accuracy_score(y_test, predicoes_rf)

## Comparação do desempenho entre modelos

In [48]:
print("Comparação entre os modelos LogisticRegression e RandomForestClassifier")
print("")
display(f"Precisão do modelo lr: {precisao_lr * 100:.2f}%")
print("==============================")
display(f"Precisão do modelo rf: {precisao_rf * 100:.2f}%")

Comparação entre os modelos LogisticRegression e RandomForestClassifier



'Precisão do modelo lr: 83.22%'

'Precisão do modelo rf: 95.04%'

## Salvando o melhor modelo para ser utilizado no Streamlit

In [50]:
joblib.dump(modelo_rf, 'modelo_rf.pkl')
print("Modelo salvo com sucesso!")

Modelo salvo com sucesso!


In [55]:
# Função para carregar o modelo RandomForestClassifier salvo e criar o pipeline
@st.cache_resource
def carregar_modelo_rf_pipeline():
    try:
        modelo_rf_carregado = joblib.load('modelo_rf.pkl')
    except FileNotFoundError:
        st.error("Arquivo do modelo 'modelo_rf.pkl' não encontrado. Por favor, verifique o caminho.")
        return None

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_features),
            ("cat", categorical_features)
        ]
    )

    # Criação do pipeline que inclui o pré-processador e o modelo carregado
    pipeline_rf = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("classifier", modelo_rf_carregado) # Usando o modelo RF carregado aqui
    ])

    return pipeline_rf

modelo_pipeline = carregar_modelo_rf_pipeline()

if modelo_pipeline is None:
    st.stop()

2025-07-10 15:43:44.150 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-10 15:43:44.153 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-10 15:43:44.154 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-10 15:43:44.155 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-10 15:43:44.199 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-10 15:43:44.200 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-10 15:43:44.203 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


# Inteface Streamlit

In [65]:
st.markdown("""
Esta aplicação utiliza Machine Learning para prever o nível de obesidade com base em características físicas e hábitos de vida.
**Use a barra lateral para inserir seus dados e obter uma previsão personalizada.** Explore o painel analítico abaixo para descobrir insights sobre os fatores que influenciam a obesidade.
""")

# --- Barra Lateral (Sidebar) para Entradas do Usuário ---
st.sidebar.header("📊 Insira seus Dados para Análise")

with st.sidebar.form("formulario_previsao"):
    st.subheader("👤 Informações Pessoais")
    # Usando colunas para um layout mais compacto
    col_genero, col_idade = st.columns(2)
    with col_genero:
        gender = st.selectbox("Gênero", dados['Gender'].unique(), index=0)
    with col_idade:
        age = st.slider("Idade", 14, 70, 25)

    col_altura, col_peso = st.columns(2)
    with col_altura:
        height = st.slider("Altura (m)", 1.40, 2.10, 1.75, format="%.2f")
    with col_peso:
        weight = st.slider("Peso (kg)", 40.0, 180.0, 80.0, format="%.1f")


    st.subheader("🍏 Hábitos Alimentares e Estilo de Vida")
    family_history = st.selectbox("Histórico familiar de sobrepeso?", ["yes", "no"], format_func=lambda x: "Sim" if x == 'yes' else "Não")
    favc = st.selectbox("Consumo frequente de alimentos calóricos (FAVC)?", ["yes", "no"], format_func=lambda x: "Sim" if x == 'yes' else "Não")
    fcvc = st.slider("Frequência de consumo de vegetais (FCVC)", 1.0, 3.0, 2.0, step=0.5, help="1: Nunca, 2: Às vezes, 3: Sempre")
    ncp = st.slider("Número de refeições principais/dia", 1.0, 4.0, 3.0, step=0.5)
    caec = st.selectbox("Consome alimentos entre as refeições (CAEC)?", dados['CAEC'].unique())
    smoke = st.selectbox("É fumante?", ["no", "yes"], format_func=lambda x: "Sim" if x == 'yes' else "Não")
    ch2o = st.slider("Consumo diário de água (Litros)", 1.0, 3.0, 2.0, step=0.1)
    scc = st.selectbox("Monitora o consumo de calorias?", ["no", "yes"], format_func=lambda x: "Sim" if x == 'yes' else "Não")
    faf = st.slider("Frequência de atividade física (dias/semana)", 0.0, 7.0, 3.0, step=0.5)
    tue = st.slider("Tempo de uso de telas (horas/dia)", 0.0, 5.0, 1.0, step=0.1)
    calc = st.selectbox("Frequência de consumo de álcool (CALC)", dados['CALC'].unique())
    mtrans = st.selectbox("Principal meio de transporte", dados['MTRANS'].unique())

    st.markdown("---") # Adicionar um separador visual
    botao_submeter = st.form_submit_button("✨ Gerar Análise e Previsão")

# --- Exibição do Resultado da Previsão ---
if botao_submeter:
    # Verificar se o pipeline e os dados foram carregados antes de fazer a previsão
    if modelo_pipeline is not None:
        input_dict = {
            "Gender": gender, "Age": age, "Height": height, "Weight": weight,
            "family_history": family_history, "FAVC": favc, "FCVC": fcvc, "NCP": ncp,
            "CAEC": caec, "SMOKE": smoke, "CH2O": ch2o, "SCC": scc, "FAF": faf,
            "TUE": tue, "CALC": calc, "MTRANS": mtrans
        }
        input_df = pd.DataFrame([input_dict])

        # Fazer a previsão usando o pipeline
        predicao = modelo_pipeline.predict(input_df)[0]
        probabilidade_predicao = modelo_pipeline.predict_proba(input_df)
        imc = weight / (height ** 2)

        st.header("🎯 Resultado da Análise Preditiva")
        col_metrica1, col_metrica2 = st.columns(2)
        with col_metrica1:
            st.metric(label="Nível de Obesidade Previsto", value=predicao)
        with col_metrica2:
            st.metric(label="Seu IMC (Índice de Massa Corporal)", value=f"{imc:.2f}")

        # Mensagem baseada na previsão
        if "Normal_Weight" in predicao:
            st.balloons()
            st.success(f"🎉 Ótima notícia! Com base nos dados fornecidos, o modelo indica uma forte tendência para **{predicao}**. Continue com seus hábitos saudáveis!")
        elif "Insufficient_Weight" in predicao:
             st.warning(f"⚠️ Atenção: Com base nos dados fornecidos, o modelo indica uma tendência para **{predicao}**. É importante buscar orientação profissional para avaliar sua saúde e nutrição.")
        elif "Overweight" in predicao:
             st.warning(f"⚠️ Atenção: Com base nos dados fornecidos, o modelo indica uma tendência para **{predicao}**. Considerar ajustes nos hábitos pode ser benéfico.")
        elif "Obesity" in predicao:
             st.error(f"🚨 Alerta: Com base nos dados fornecidos, o modelo indica uma forte tendência para **{predicao}**. É altamente recomendado procurar acompanhamento médico e nutricional.")
        else:
            st.info(f"Com base nos dados fornecidos, o modelo indica uma tendência para **{predicao}**.")


        st.subheader("Probabilidade por Categoria")
        df_prob = pd.DataFrame(probabilidade_predicao, columns=modelo_pipeline.classes_, index=["Probabilidade"]).T
        df_prob = df_prob.sort_values(by="Probabilidade", ascending=False)

        fig_prob = px.bar(
            df_prob, x=df_prob.index, y='Probabilidade',
            labels={'Probabilidade': 'Probabilidade', 'index': 'Nível de Obesidade'},
            text_auto='.2%', title="Probabilidade para Cada Nível de Obesidade"
        )
        fig_prob.update_layout(yaxis_title="Probabilidade", xaxis_title="Nível de Obesidade")
        st.plotly_chart(fig_prob, use_container_width=True)
    else:
        st.error("O pipeline do modelo não foi carregado corretamente. Por favor, verifique as etapas anteriores.")


# --- Seção de Insights Analíticos ---
# Verificar se os dados brutos foram carregados antes de exibir as visualizações
if dados is not None:
    st.markdown("<hr>", unsafe_allow_html=True)
    st.header("📊 Painel Analítico Interativo")
    st.markdown("Explore as relações entre diferentes fatores e os níveis de obesidade no conjunto de dados.")

    # Usando expansores para organizar as visualizações
    with st.expander("Distribuição Geral e Fatores Relevantes"):
        col1, col2 = st.columns(2)

        with col1:
            st.subheader("Distribuição Geral da Obesidade")
            contagem_obesidade = dados['Obesity'].value_counts()
            fig_pizza = px.pie(names=contagem_obesidade.index, values=contagem_obesidade.values,
                                title="Proporção de cada Nível de Obesidade", hole=0.3)
            fig_pizza.update_layout(legend_title_text='Níveis de Obesidade')
            st.plotly_chart(fig_pizza, use_container_width=True)

        with col2:
            st.subheader("Fatores Mais Relevantes na Previsão")
            # Verificar se modelo_pipeline está disponível e tem o atributo feature_importances_
            if modelo_pipeline is not None and hasattr(modelo_pipeline.named_steps['classifier'], 'feature_importances_'):
                importances = modelo_pipeline.named_steps['classifier'].feature_importances_
                # Acessando os nomes das features após o pré-processamento usando get_feature_names_out()
                preprocessor = modelo_pipeline.named_steps['preprocessor']
                # Certifique-se de que o preprocessor foi ajustado (o que acontece quando o pipeline é ajustado)
                try:
                    feature_names_out = preprocessor.get_feature_names_out()
                except Exception as e:
                    st.error(f"Erro ao obter nomes das features do pré-processador: {e}")
                    feature_names_out = None # Definir como None em caso de erro


                if feature_names_out is not None:
                    df_importancia = pd.DataFrame({'Fator': feature_names_out, 'Importância': importances})
                    df_importancia = df_importancia.sort_values(by='Importância', ascending=False).head(15)

                    fig_importancia = px.bar(
                        df_importancia, x='Importância', y='Fator', orientation='h',
                        title='Importância de Cada Fator para o Modelo',
                        labels={'Importância': 'Importância Relativa', 'Fator': 'Fator'},
                        text='Importância'
                    )
                    fig_importancia.update_traces(texttemplate='%{text:.2f}', textposition='outside')
                    fig_importancia.update_layout(yaxis={'categoryorder':'total ascending'})
                    st.plotly_chart(fig_importancia, use_container_width=True)
                else:
                     st.warning("Não foi possível exibir a importância dos fatores devido a um erro ao obter os nomes das features processadas.")

            else:
                st.warning("Não foi possível exibir a importância dos fatores. Verifique se o modelo pipeline foi carregado e se o classificador possui o atributo 'feature_importances_'.")


    with st.expander("Análise Demográfica e de Hábitos"):
        col3, col4 = st.columns(2)

        with col3:
            st.subheader("Distribuição de Idade por Nível de Obesidade")
            fig_boxplot_idade = px.box(dados, x="Obesity", y="Age", color="Obesity",
                                        title="Faixa Etária por Categoria de Obesidade",
                                        labels={"Obesity": "Nível de Obesidade", "Age": "Idade"})
            st.plotly_chart(fig_boxplot_idade, use_container_width=True)

        with col4:
            st.subheader("Distribuição de Obesidade por Gênero")
            obesidade_genero = dados.groupby(['Gender', 'Obesity']).size().reset_index(name='Contagem')
            fig_barras_genero = px.bar(
                obesidade_genero, x="Obesity", y="Contagem", color="Gender", barmode="group",
                title="Contagem de Níveis de Obesidade por Gênero",
                labels={"Obesity": "Nível de Obesidade", "Contagem": "Número de Pessoas", "Gender": "Gênero"}
            )
            st.plotly_chart(fig_barras_genero, use_container_width=True)

    with st.expander("Relações entre Fatores Físicos e Obesidade"):
        st.subheader("Relação entre Peso, Altura e Nível de Obesidade")
        fig_dispersao = px.scatter(
            dados, x="Height", y="Weight", color="Obesity",
            hover_data=['Age', 'Gender'], title="Dispersão de Peso vs. Altura",
            labels={"Height": "Altura (m)", "Weight": "Peso (kg)", "Obesity": "Nível de Obesidade"}
        )
        st.plotly_chart(fig_dispersao, use_container_width=True)

        st.subheader("Matriz de Correlação entre Variáveis Numéricas")
        st.markdown("Esta matriz mostra como as variáveis numéricas se relacionam. Valores próximos de 1 (vermelho) ou -1 (azul) indicam forte correlação.")
        df_numerico = dados.select_dtypes(include=[np.number])
        correlacao = df_numerico.corr()

        fig_heatmap = px.imshow(
            correlacao, text_auto=True, aspect="auto",
            color_continuous_scale='RdBu_r', title="Mapa de Calor das Correlações"
        )
        st.plotly_chart(fig_heatmap, use_container_width=True)

else:
    st.error("Os dados brutos não foram carregados corretamente para o painel analítico.")

2025-07-10 18:11:54.682 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-10 18:11:54.683 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-10 18:11:54.722 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-10 18:11:54.737 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-10 18:11:54.763 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-10 18:11:54.770 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-10 18:11:54.780 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-10 18:11:54.786 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar